# Scraping Houses in Redfin

In [ ]:
print("John Spinelli, U50128653")

**To be completed INDIVIDUALLY and due on April 10 at 7pm.**

In this homework, we will practice web scraping. Let's get some basic information for each house/apartment in Massachusetts.

You can see houses around Boston University [here](https://www.redfin.com/zipcode/02215)

![Information to be scraped](redfin.png)


On each house page, scrape the information of this [house](https://www.redfin.com/MA/Boston/120-Mountfort-St-02215/unit-502/home/11838719), including 
* full address
* price
* number of beds
* number of baths (full or not)
* Sq. Ft (square feet)
* price per Sq. Ft
* property type
* County
* community
* built
* Property details (e.g. dishwasher, elevator, parking, heating, air conditioning, maintenance, etc.)

Try to extract as many features as possible. You will use this information in future homework. 


Save the data in "house.csv" in the following format:

full address, price, beds, baths, ......

**To receive credit, you must commit house.csv to Github.** **(20 pts)**

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Apr 10 23:18:35 2017

@author: JohnSpinelli
"""

__author__ = 'JohnSpinelli'

import csv
import numpy as np
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import random 
import urllib.request
import re
import pandas as pd
from pandas import DataFrame as Df



prefix = 'https://www.redfin.com'
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36"
headers = { 'User-Agent' : user_agent }

path_to_chromedriver = "/Users/JohnSpinelli/desktop/chromedriver"
driver = webdriver.Chrome(executable_path = path_to_chromedriver)
randTimer = random.uniform(1,4)
time.sleep(randTimer)
driver.get('https://www.redfin.com/zipcode/02215')
html = driver.page_source
soup = BeautifulSoup(html,"html")

# Retrieve the URLs of the 6 listings

propertyUrls = []

links = soup.find_all("a", class_="link",href=True)
for item in links:
    propertyUrls.append(item.get("href"))
   
propertyUrls = list(set(propertyUrls))
propertyUrls = [prefix + x for x in propertyUrls]



# Try a property URL and extract the information needed and store in a dict
propertyList = []
for url in propertyUrls:
    
    randTimer = random.uniform(1,4)
    time.sleep(randTimer)
    html2 = requests.get(url,headers=headers)
    time.sleep(randTimer)
    soup2 = BeautifulSoup(html2.text, 'html')
            
        
    # Div that contains address, price, beds, bath, sqft/sqft price
             
    address = soup2.find('span', {"class":"street-address"})['title']
    mainStats = soup2.find_all('div',{'class':'main-font statsValue'})
    price = mainStats[0].get_text()
    beds = mainStats[1].get_text()
    baths = mainStats[2].get_text()
    #sqftStats = soup2.find_all('div',{'class':'info-block left-divider sqft'})
    
    sqft = soup2.find_all('span',{'class':'main-font statsValue'})[0].get_text()
    sqftPrice = soup2.find_all('div',{'class':'statsLabel'})[0].get_text().split(' ')[0]
    
    # Div that contains property type, community, county, community, built
    
    keyDetails = soup2.find_all('div', {'class':'keyDetail'})
    propertyType = keyDetails[1].get_text().split('Type')[1]
    community = keyDetails[3].get_text().split('Community')[1]
    county = keyDetails[4].get_text().split('County')[1]
    built = keyDetails[6].get_text().split('Built')[1]

    propertyDict = {'Address':address,'Price':price,'Beds':beds,\
                         'Baths':baths,'Square Feet':sqft,'Price Per Sqft':sqftPrice,\
                         'Property Type':propertyType,'Community':community,\
                         'County':county,'Built':built}
    
# Div that contains property details
    
    propertyDetails = soup2.find_all('div', {'class':'amenity-group'})
    details = []
    columns = []
    for i in range(1,6):
        x =re.findall('[A-Z][^A-Z]*',propertyDetails[i].get_text())
        details.append(' '.join(x))
        
    propertyDict['Detail 1'] = details[0]
    propertyDict['Detail 2'] = details[1]
    propertyDict['Detail 3'] = details[2]
    propertyDict['Detail 4'] = details[3]
    propertyDict['Detail 5'] = details[4]
              
    
    propertyList.append({'Address':address,'Price':price,'Beds':beds,\
                         'Baths':baths,'Square Feet':sqft,'Price Per Sqft':sqftPrice,\
                         'Property Type':propertyType,'Community':community,\
                         'County':county,'Built':built,'Detail 1':propertyDict['Detail 1'],\
                         'Detail 2':propertyDict['Detail 2'],'Detail 3':propertyDict['Detail 3'],\
                         'Detail 4':propertyDict['Detail 4'],\
                         'Detail 5':propertyDict['Detail 5']})
    

listingDf = Df(propertyList,columns=['Address','Price','Beds','Baths','Square Feet',\
               'Price Per Sqft', 'Property Type', 'Community',\
               'County','Built','Detail 1','Detail 2','Detail 3','Detail 4','Detail 5'])

csvPath = '/Users/JohnSpinelli/desktop/house.csv'
listingDf.to_csv(csvPath)









-------

Now that you can scrape and extract information of a house, scrape the information of houses in Massachusetts (as of today there are around 10000 houses in MA listed in Redfin). To see all the listings, you can query all the zip codes in Massachusetts. You can see all the zip codes in Massachusetts [here](https://github.com/evimaria/CS506-Spring2007/blob/master/Homeworks/zipcodes.txt).
Save the data in "MA_houses.csv", one line for each house:
**To receive credit, you must commit MA_houses.csv to Github** **(20 pts)**

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Apr 10 23:18:35 2017

@author: JohnSpinelli
"""

__author__ = 'JohnSpinelli'

import csv
import numpy as np
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import random 
import urllib.request
import re
import pandas as pd
from pandas import DataFrame as Df


zipFile = '/Users/JohnSpinelli/desktop/zipcodes.txt'
with open(zipFile) as f: 
    zips = f.readlines()
zips = [x.strip() for x in zips]

for zip1 in zips:
    prefix = 'https://www.redfin.com'
    user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36"
    headers = { 'User-Agent' : user_agent }
    
    path_to_chromedriver = "/Users/JohnSpinelli/desktop/chromedriver"
    driver = webdriver.Chrome(executable_path = path_to_chromedriver)
    randTimer = random.uniform(1,4)
    time.sleep(randTimer)
    driver.get('https://www.redfin.com/zipcode/' + zip1)
    html = driver.page_source
    soup = BeautifulSoup(html,"html")
    
    # Retrieve the URLs of the 6 listings
    driver.close()
    propertyUrls = []
    
    
    links = soup.find_all("a", class_="link",href=True)
    for item in links:
        propertyUrls.append(item.get("href"))
    
    if (len(propertyUrls) > 1):
        propertyUrls = list(set(propertyUrls))
        propertyUrls = [prefix + x for x in propertyUrls]
        
        
        
        # Try a property URL and extract the information needed and store in a dict
        propertyList = []
        for url in propertyUrls:
            
            randTimer = random.uniform(1,4)
            time.sleep(randTimer)
            html2 = requests.get(url,headers=headers)
            time.sleep(randTimer)
            soup2 = BeautifulSoup(html2.text, 'html')
                    
                
            # Div that contains address, price, beds, bath, sqft/sqft price
                     
            address = soup2.find('span', {"class":"street-address"})['title']
            mainStats = soup2.find_all('div',{'class':'main-font statsValue'})
            try:
                price = mainStats[0].get_text()
                beds = mainStats[1].get_text()
                baths = mainStats[2].get_text()
            except:
                pass
            
            try:
                sqft = soup2.find_all('span',{'class':'main-font statsValue'})[0].get_text()
                sqftPrice = soup2.find_all('div',{'class':'statsLabel'})[0].get_text().split(' ')[0]
            except:
                pass
            # Div that contains property type, community, county, community, built
            
            keyDetails = soup2.find_all('div', {'class':'keyDetail'})
            try:
                propertyType = keyDetails[1].get_text().split('Type')[1]
                community = keyDetails[3].get_text().split('Community')[1]
                county = keyDetails[4].get_text().split('County')[1]
                built = keyDetails[6].get_text().split('Built')[1]
        
                propertyDict = {'Address':address,'Price':price,'Beds':beds,\
                                 'Baths':baths,'Square Feet':sqft,'Price Per Sqft':sqftPrice,\
                                 'Property Type':propertyType,'Community':community,\
                                 'County':county,'Built':built}
            
        # Div that contains property details
            
                propertyDetails = soup2.find_all('div', {'class':'amenity-group'})
                details = []
                columns = []
                for i in range(1,6):
                    x =re.findall('[A-Z][^A-Z]*',propertyDetails[i].get_text())
                    details.append(' '.join(x))
                    
                propertyDict['Detail 1'] = details[0]
                propertyDict['Detail 2'] = details[1]
                propertyDict['Detail 3'] = details[2]
                propertyDict['Detail 4'] = details[3]
                propertyDict['Detail 5'] = details[4]
                          
                
                propertyList.append({'Address':address,'Price':price,'Beds':beds,\
                                     'Baths':baths,'Square Feet':sqft,'Price Per Sqft':sqftPrice,\
                                     'Property Type':propertyType,'Community':community,\
                                     'County':county,'Built':built,'Detail 1':propertyDict['Detail 1'],\
                                     'Detail 2':propertyDict['Detail 2'],'Detail 3':propertyDict['Detail 3'],\
                                     'Detail 4':propertyDict['Detail 4'],\
                                     'Detail 5':propertyDict['Detail 5']})
            except:
                pass

listingDf = Df(propertyList,columns=['Address','Price','Beds','Baths','Square Feet',\
               'Price Per Sqft', 'Property Type', 'Community',\
               'County','Built','Detail 1','Detail 2','Detail 3','Detail 4','Detail 5'])

csvPath = '/Users/JohnSpinelli/desktop/MA_houses.csv'
listingDf.to_csv(csvPath)
      

Attempted to retrieve all of BHousing but could not stop from being blocked. 

Note that many websites will block you if you are sending too many queries. To prevent getting block, you should wait between queries (typically 2-5 seconds). 

-------